# Abstract Algebras

## References

* [Group Explorer](https://nathancarter.github.io/group-explorer/index.html) -- Visualization software for the abstract algebra classroom
* [Groupprops, The Group Properties Wiki (beta)](https://groupprops.subwiki.org/wiki/Main_Page)
* [Klein four-group, V4](https://en.wikipedia.org/wiki/Klein_four-group)
* [Cyclic group](https://en.wikipedia.org/wiki/Cyclic_group)

## Algebra Definition

In [39]:
import itertools as it
import json
import os

In [40]:
aa_path = os.path.join(os.getenv('PYPROJ'), 'abstract_algebra')
aa_path

'/Users/alfredreich/Documents/Github/repo/abstract_algebra'

In [41]:
alg_dir = os.path.join(aa_path, "Algebras")
alg_dir

'/Users/alfredreich/Documents/Github/repo/abstract_algebra/Algebras'

In [42]:
v4_json = os.path.join(alg_dir, "v4_klein_4_group.json")
v4_json

'/Users/alfredreich/Documents/Github/repo/abstract_algebra/Algebras/v4_klein_4_group.json'

In [43]:
!cat {v4_json}

{"type": "Group",
 "name": "V4",
 "desc": "Klein-4 group",
 "elements": ["e", "h", "v", "hv"],
 "table": [[0, 1, 2, 3],
           [1, 0, 3, 2],
           [2, 3, 0, 1],
           [3, 2, 1, 0]]
}


In [48]:
class Algebra:
    '''An abstract algbra with a finite number of elements and an addition table.'''
    
    def __init__(self, name, description, element_names, addition_table):
        self.name = name
        self.desc = description
        self.elements = element_names
        self.table = addition_table
        # For efficiency, calculate the headers up front
        self.col_header = self.table[0]
        self.row_header = [row[0] for row in self.table]
    
    @classmethod
    def make(cls, *args):
        '''Create an algebra from a JSON file or a Python dictionary.'''

        if str(args[0]):
            print(args[0])
            with open(args[0], 'r') as fin:
                alg_dict = json.load(fin)
        else:
            alg_dict = args[0]
        
        alg = cls(alg_dict['name'],
                  alg_dict['desc'],
                  alg_dict['elements'],
                  alg_dict['table'])

        return alg
        
    def __str__(self):
        return f"<{self.__class__.__name__}: {self.name}, {self.desc}>"
    
    def __repr__(self):
        return f"{self.__class__.__name__}('{self.name}', '{self.desc}', {self.elements}, {self.table})"
    
    def to_dict(self):
        return {'type': self.__class__.__name__,
                'name': self.name,
                'desc': self.desc,
                'elements': self.elements,
                'table': self.table}
    
    def inverse(self, element):
        elem_index = self.elements.index(element)
        row_index = self.row_header.index(elem_index)
        col_index = self.table[row_index].index(0)
        return self.elements[self.col_header[col_index]]
    
    def addition_table_with_names(self):
        return [[self.elements[x] for x in row] for row in self.table]
        
    def add(self, r, c):
        '''Return the sum of elements, r & c.
        The inputs, r & c, can be numbers or strings, but if either
        input is a number, then a number will be returned, otherwise
        the sum's element name (a string) will be returned.'''

        # Table lookup requires numbers
        r_ = r; c_ = c
        str_result = False
        if type(r) == str:
            r_ = self.elements.index(r)
            str_result = True
        if type(c) == str:
            c_ = self.elements.index(c)
            str_result = True
        
        # Lookup the product based on the row & column indices
        row_index = self.row_header.index(r_)
        col_index = self.col_header.index(c_)
        product = self.table[row_index][col_index]
        
        # If either input value was a string, then return a string,
        # otherwise return a number
        if str_result:
            return self.elements[product]
        else:
            return product
        
    def __mul__(self, other):
        '''Return the direct product of this algebra with the input algebra, other.'''
        new_name = self.name + "_x_" + other.name
        new_desc = "Direct product of " + self.name + " & " + other.name
        new_elements = list(it.product(self.elements, other.elements))
        new_table = list()
        for e in new_elements:
            new_row = list()
            for f in new_elements:
                new_row.append(new_elements.index((self.add(e[0], f[0]), other.add(e[1], f[1]))))
            new_table.append(new_row)
        return self.__class__(new_name,
                              new_desc,
                              list([f"{c[0]},{c[1]}" for c in new_elements]),
                              new_table)
    
    def swap(self, a, b):
        '''Change the algbra's definition by swapping the order of two elements, a & b.'''
        elem = self.elements
        i, j = elem.index(a), elem.index(b)
        elem[j], elem[i] = elem[i], elem[j]
        for row in self.table:
            k, m = row.index(i), row.index(j)
            row[k], row[m] = row[m], row[k]
        return None
    
    def elements(self):
        return self.table[0]

In [50]:
foo = Algebra.make(v4_json)
foo

/Users/alfredreich/Documents/Github/repo/abstract_algebra/Algebras/v4_klein_4_group.json


Algebra('V4', 'Klein-4 group', ['e', 'h', 'v', 'hv'], [[0, 1, 2, 3], [1, 0, 3, 2], [2, 3, 0, 1], [3, 2, 1, 0]])

# Utilities

<b>make-table</b> helps turn the XML-based tables at Groupprops into a list of lists for use here.

INSTRUCTIONS FOR USE:
1. Copy the table from there and paste it here
1. Find & Replace the strings, "\<row\>" and "\<\/row\>", with nothing
1. Place triple quotes around the result and give it a variable name

In [ ]:
def make_table(table_string):
    return [[int(n) for n in row.strip().split(" ")]
            for row in table_string.splitlines()]

def swap_list_items(lst, item1, item2):
    a, b = lst.index(item1), lst.index(item2)
    lst[b], lst[a] = lst[a], lst[b]
    return None

## Group Definition

In [ ]:
class Group(Algebra):
    pass

### Klein-4 Group

See this [definition at GitHub](https://github.com/nathancarter/group-explorer/blob/master/groups/V_4.group).

In [ ]:
v4 = Group('V4',
           'Klein-4 group',
           ['e',  'h',  'v', 'hv'],
           [[0, 1, 2, 3],
            [1, 0, 3, 2],
            [2, 3, 0, 1],
            [3, 2, 1, 0]]
          )

In [ ]:
v4.addition_table_with_names()

In [ ]:
v4.add('h','v')

In [ ]:
for elem in v4.elements:
    print(f"inv({elem}) = {v4.inverse(elem)}")

In [ ]:
print(v4)

In [ ]:
v4json = json.dumps(v4.to_dict())
v4json

In [ ]:
path = "/Users/alfredreich/Documents/Github/repo/abstract_algebra/Algebras"

### Cyclic group of order 4

See this [definition at GitHub](https://github.com/nathancarter/group-explorer/blob/master/groups/Z_4.group).

In [ ]:
z4 = Group('Z4',
           'Cyclic group of order 4',
           ['e', 'a', 'a^2', 'a^3'],
           [[0, 1, 2, 3],
            [1, 2, 3, 0],
            [2, 3, 0, 1],
            [3, 0, 1, 2]]
          )

In [ ]:
z4.addition_table_with_names()

In [ ]:
print(z4)

In [ ]:
z4.add(2,2)

In [ ]:
z4.add('a^2', 'a^3')

In [ ]:
for elem in z4.elements:
    print(f"inv({elem}) = {z4.inverse(elem)}")

### Symmetric group on 3 letters

See this [definition at GitHub](https://github.com/nathancarter/group-explorer/blob/master/groups/S_3.group). "Another name for this group is "Dihedral group on 3 vertices."

In [ ]:
s3 = Group('S3',
           'Symmetric group on 3 letters',
           ['e', 'r', 'r^2', 'f', 'fr', 'rf'],
           [[0, 1, 2, 3, 4, 5],
            [1, 2, 0, 5, 3, 4],
            [2, 0, 1, 4, 5, 3],
            [3, 4, 5, 0, 1, 2],
            [4, 5, 3, 2, 0, 1],
            [5, 3, 4, 1, 2, 0]]
          )

In [ ]:
s3.addition_table_with_names()

In [ ]:
print(s3)

In [ ]:
s3.add('fr', 'r^2')

In [ ]:
for elem in s3.elements:
    print(f"inv({elem}) = {s3.inverse(elem)}")

In [ ]:
s3

### Symmetric Group, S3 (not same as S3, above)

This is the [Symmetric group, S3, as specified at Groupprops](https://groupprops.subwiki.org/wiki/Symmetric_group:S3).

W.r.t., how to interpret the addition table, the following quote helps:

[[from Wikipedia](https://groupprops.subwiki.org/wiki/Symmetric_group:S3)] "We portray elements as permutations on the set \{ 1,2,3 \} using the cycle decomposition. The row element is added on the left and the column element on the right, with the assumption of functions written on the left. This means that the column element is applied first and the row element is applied next."

In [ ]:
s3x = Group('S3X',
           'Another version of the symmetric group on 3 letters',
           ['()', '(1,2)', '(2,3)', '(1,3)', '(1,2,3)', '(1,3,2)'],
           [[0, 1, 2, 3, 4, 5],
            [1, 0, 4, 5, 2, 3],
            [2, 5, 0, 4, 3, 1],
            [3, 4, 5, 0, 1, 2],
            [4, 3, 1, 2, 5, 0],
            [5, 2, 3, 1, 0, 4]]
          )

In [ ]:
s3x.addition_table_with_names()

In [ ]:
s3x.add('(1,2)', '(2,3)')

In [ ]:
for elem in s3x.elements:
    print(f"inv({elem}) = {s3x.inverse(elem)}")

## Z_2 x Z_2 x Z_2

In [ ]:
Z2xZ2xZ2 = Group('Z_2 x Z_2 x Z_2',
                 'no description',
                 ['eee', 'aee', 'eae', 'aae', 'eea', 'aea', 'eaa', 'aaa'],
                 [[0, 1, 2, 3, 4, 5, 6, 7],
                  [1, 0, 3, 2, 5, 4, 7, 6],
                  [2, 3, 0, 1, 6, 7, 4, 5],
                  [3, 2, 1, 0, 7, 6, 5, 4],
                  [4, 5, 6, 7, 0, 1, 2, 3],
                  [5, 4, 7, 6, 1, 0, 3, 2],
                  [6, 7, 4, 5, 2, 3, 0, 1],
                  [7, 6, 5, 4, 3, 2, 1, 0]]
                )

In [ ]:
Z2xZ2xZ2.addition_table_with_names()

In [ ]:
for elem in Z2xZ2xZ2.elements:
    print(f"inv({elem}) = {Z2xZ2xZ2.inverse(elem)}")

### Example

In [ ]:
foo = '''         0 1 2 3 4 5 6 7 
         1 0 3 2 5 4 7 6 
         2 3 0 1 6 7 4 5 
         3 2 1 0 7 6 5 4 
         4 5 6 7 0 1 2 3 
         5 4 7 6 1 0 3 2 
         6 7 4 5 2 3 0 1 
         7 6 5 4 3 2 1 0 '''

In [ ]:
z2_x_z2_x_z2 = make_table(foo)
z2_x_z2_x_z2

In [ ]:
fu = "<e,e,e> <a,e,e> <e,a,e> <a,a,e> <e,e,a> <a,e,a> <e,a,a> <a,a,a>"

In [ ]:
fubar = fu.split(" ")
fubar

## Direct Products

In [ ]:
z2 = Group('Z2',
           'Cyclic group of order 2',
           ['E', 'A'],
           [[0, 1],
            [1, 0]]
          )

z3 = Group('Z3',
           'Cyclic group of order 3',
           ['E', 'A', 'B'],
           [[0, 1, 2],
            [1, 2, 0],
            [2, 0, 1]]
          )

In [ ]:
def direct_product(g1, g2):
    new_elements = list(it.product(g1.elements, g2.elements))
    new_table = list()
    for e1 in new_elements:
        new_row = list()
        for e2 in new_elements:
            new_row.append(new_elements.index((g1.add(e1[0], e2[0]), g2.add(e1[1], e2[1]))))
        new_table.append(new_row)
    return list([f"{c[0]},{c[1]}" for c in new_elements])

In [ ]:
z3_x_z3 = z3 * z3
z3_x_z3

In [ ]:
z3_x_z3.elements

In [ ]:
z3_x_z3.table

In [ ]:
for elem in z3_x_z3.elements:
    print(f"inv({elem}) = {z3_x_z3.inverse(elem)}")

In [ ]:
z2_x_z2_x_z2 = z2 * z2 * z2

In [ ]:
z2_x_z2_x_z2.elements

In [ ]:
z2_x_z2_x_z2.table

In [ ]:
for elem in z2_x_z2_x_z2.elements:
    print(f"inv({elem}) = {z2_x_z2_x_z2.inverse(elem)}")